In [1]:
import os
drive_path = '/media/stephen/Brain20191/allen-brain-observatory/visual-coding-2p/'
if not os.path.isdir(drive_path):
    drive_path = '/media/alihamodi/Brain2019/allen-brain-observatory/visual-coding-2p/'
    
import warnings; warnings.simplefilter('ignore')
import os
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from utils.data_wrappers import *
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)

from utils.polarplot import *
from utils.polardata import *
from utils.data_wrappers import *

ModuleNotFoundError: No module named 'allensdk'

In [46]:
visual_area = 'VISp'
cre_line ='Cux2-CreERT2'
containers = boc.get_experiment_containers(targeted_structures=[visual_area],
                                           cre_lines=[cre_line],
                                            imaging_depths = [175, 275]
                                          )
ec_id = [container["id"] for container in containers]
#ec_id = 511510650
exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS],
                                experiment_container_ids=ec_id,
        targeted_structures = [visual_area],
        imaging_depths = [175, 275],
        cre_lines = [cre_line])

In [47]:
containers

[{'id': 511510736,
  'imaging_depth': 175,
  'targeted_structure': 'VISp',
  'cre_line': 'Cux2-CreERT2',
  'reporter_line': 'Ai93(TITL-GCaMP6f)',
  'donor_name': '222426',
  'specimen_name': 'Cux2-CreERT2;Camk2a-tTA;Ai93-222426',
  'tags': [],
  'failed': False},
 {'id': 511510855,
  'imaging_depth': 175,
  'targeted_structure': 'VISp',
  'cre_line': 'Cux2-CreERT2',
  'reporter_line': 'Ai93(TITL-GCaMP6f)',
  'donor_name': '229106',
  'specimen_name': 'Cux2-CreERT2;Camk2a-tTA;Ai93-229106',
  'tags': [],
  'failed': False},
 {'id': 511509529,
  'imaging_depth': 175,
  'targeted_structure': 'VISp',
  'cre_line': 'Cux2-CreERT2',
  'reporter_line': 'Ai93(TITL-GCaMP6f)',
  'donor_name': '222420',
  'specimen_name': 'Cux2-CreERT2;Camk2a-tTA;Ai93-222420',
  'tags': [],
  'failed': False},
 {'id': 511507650,
  'imaging_depth': 175,
  'targeted_structure': 'VISp',
  'cre_line': 'Cux2-CreERT2',
  'reporter_line': 'Ai93(TITL-GCaMP6f)',
  'donor_name': '222424',
  'specimen_name': 'Cux2-CreERT2;Cam

In [32]:
# polardata = get_avg_normalized_response(
#         boc=boc,
#         session_id=exp_id,
#         cell_specimen_id = cell_specimen_id,
#         temporal_frequency=2.0)

In [48]:
exps = pd.DataFrame.from_dict(exps)
cells = boc.get_cell_specimens()
cells = pd.DataFrame.from_records(cells)
cells = cells[cells["experiment_container_id"].isin(set(ec_id))]

In [43]:
# exps = boc.get_ophys_experiments(
#     experiment_container_ids=[ec_id],
#     stimuli=[stim_info.DRIFTING_GRATINGS])
# exps = pd.DataFrame.from_dict(exps)
# cells = boc.get_cell_specimens()
# cells = pd.DataFrame.from_records(cells)

dg_response_filter = get_dg_response_filter_from_saskia()
# polardata_df = polardata.to_frame()

# in_category_cells = cells[cells.experiment_container_id == ec_id]
# responsive_cells = get_filtered_cells(in_category_cells, dg_response_filter)
# responsive_cells = responsive_cells[responsive_cells['p_dg'] < 0.05]
# # direction selective cells
# dsi_cells_one = responsive_cells[(responsive_cells['dsi_dg'] > 0.9)]
# dsi_cells_two = responsive_cells[(responsive_cells['dsi_dg'] > 0.8)]
# dsi_cells_three = responsive_cells[(responsive_cells['dsi_dg'] > 0.7)]
# dsi_cells_four = responsive_cells[(responsive_cells['dsi_dg'] > 0.6)]
# dsi_cells_five = responsive_cells[(responsive_cells['dsi_dg'] > 0.5)]
# print("direction-selective cells >0.9: %d" % len(dsi_cells_one))
# print("direction-selective cells >0.8: %d" % len(dsi_cells_two))
# print("direction-selective cells >0.7: %d" % len(dsi_cells_three))
# print("direction-selective cells >0.6: %d" % len(dsi_cells_four))
# print("direction-selective cells >0.5: %d" % len(dsi_cells_five))
# responsive_cells = dsi_cells_two

# in_category_cells = cells[cells.experiment_container_id == ec_id]
filtered_cells = get_filtered_cells(cells, dg_response_filter)
# need to first make dictionary to map exp containers to experiment ids

new_col = []
for index, row in filtered_cells.iterrows():
    print(index, row)
    polardata_for_this_cell = get_avg_normalized_response(
        boc = boc,
        session_id = exp_id,
        cell_specimen_id = row.cell_specimen_id,
        temporal_frequency = 2.0)
    new_dsi_cell = get_new_dsi(polardata_for_this_cell.to_frame())
    new_col.append(new_dsi_cell)
filtered_cells['new_dsi_cell'] = new_col
filtered_cells = filtered_cells[filtered_cells.new_dsi_cell > 0.3]

print ("In this experiment, we have", len(in_category_cells),
       "cells, but after filtering, we have", len(filtered_cells), "cells")


2 all_stim                                                           False
area                                                                VISp
cell_specimen_id                                               517406391
donor_full_genotype    Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...
dsi_dg                                                          0.877959
                                             ...                        
tld2_id                                                      1.77837e+08
tld2_name                                                     Camk2a-tTA
tlr1_id                                                        265943423
tlr1_name                                             Ai93(TITL-GCaMP6f)
responsive                                                          True
Name: 2, Length: 61, dtype: object


TypeError: %d format: a number is required, not Series

In [42]:
session_id = exps['id'],
session_id

(0     704298735
 1     503109347
 2     524691284
 3     501574836
 4     645413759
 5     510514474
 6     501704220
 7     502205092
 8     501271265
 9     702934964
 10    501836392
 11    510517131
 12    660513003
 13    712178511
 14    502115959
 15    502608215
 Name: id, dtype: int64,)

In [ ]:
thetas = []
rs = []
cell_ids = []
for cell_specimen_id in filtered_cells.cell_specimen_id:
    one_cell_polar_data_dict = get_avg_normalized_response(
        boc=boc,
        session_id=exp_id,
        cell_specimen_id = cell_specimen_id,
        temporal_frequency=2.0)
    if one_cell_polar_data_dict is None:
        continue
    theta, r = convert_polar_dict_to_arrays(one_cell_polar_data_dict)
    thetas.append(theta)
    rs.append(r)
    cell_ids.append(cell_specimen_id)

In [ ]:
agg_theta, agg_r = get_avg_polar(thetas, rs)
# Or use get_median_polar

fig, axes = plt.subplots(nrows=1, ncols=1, subplot_kw=dict(polar=True))
polar_plot_population(ax=axes,
                      bold_theta = agg_theta,
                      bold_r = agg_r,
                      grey_thetas = thetas, 
                      grey_rs = rs)